# <font color='brown'>楼 + 机器学习实战</font>

# 挑战：使用 RNN 完成诗词创作

## 挑战介绍

上一个实验中，我们学习了循环神经网络 RNN 的相关知识，也搭建了循环神经网络完成了情感分析的任务。同时，实验进一步了解了经典循环神经网络结构。本次挑战中，我们需要把学到的知识运用起来，通过利用循环神经网络这个强大的工具，完成诗词创作。

## 挑战知识点

- Keras 运用
- 循环神经网络实践

<div style="color: #999;font-size: 12px;">* 本挑战项目参考 [ioiogoo/poetry_generator_Keras](https://github.com/ioiogoo/poetry_generator_Keras)，并对部分内容进行改进。</div>

---

## 挑战内容

循环神经网络在自然语言处理有着极为广泛的运用。像文本，语音这样的序列模型，使用循环神经网络处理再顺心不过了。今天的挑战中，将利用从网络爬取到的古诗词做为训练集，让循环神经网络学习这些诗词后，根据提示写出诗句。

首先，下载本次挑战所使用到的文件：

In [ ]:
!wget http://labfile.oss.aliyuncs.com/courses/1081/rnn_poetry.zip # 下载数据集
!unzip rnn_poetry.zip # 解压数据集

### 数据预处理

挑战使用到的数据集来源于网络，包含有 2 万余首古诗词。我们随意截取其中一首如下：

<div style="text-align: center; color:brown;">
襄阳行乐处，歌舞白铜鞮。<br>江城回渌水，花月使人迷。<br>山公醉酒时，酩酊襄阳下。<br>头上白接篱，倒着还骑马。<br>岘山临汉江，水渌沙如雪。<br>上有堕泪碑，青苔久磨灭。<br>且醉习家池，莫看堕泪碑。<br>山公欲上马，笑杀襄阳儿。
</div>

如上所示，数据集中都是十分有意境的古诗词。对于文本数据，代码是无法像我们人一样直接看懂。所以，这就需要一些向量化、数值化的手段将其处理成机器可以识别的输入。下面，我们通过挑战提供的 `preprocess_file()` 方法对预料进行预处理，这里只保留五言绝句的预料以保证后面输出稳定。

经过文本预处理之后，依次返回：字到序号映射字典、序号到字映射字典、数据集中全部单字、数据集全文。

In [ ]:
from data_clean import preprocess_file

# 依次返回：字到序号映射字典、序号到字映射字典、数据集中全部单字、数据集全文
word2num, num2word, words, files_content = preprocess_file("poetry.txt")

把单个字处理成**序号和字**的对应关系，每一首诗词就可以由文字表示变为数字表示，这样才能作为输入被用于训练 RNN 网络。你可以自行输出字典键值查看，以便于更加熟悉数据集，例如：

In [ ]:
num2word[10], num2word[20], num2word[30]

### 建立 LSTM 模型

接下来，我们构建 LSTM 结构的循环神经网络模型。模型结构如下：

- LSTM 层：输出 `512`，输入形状为 `(6, len(words))`，设置 `return_sequences=True`。
- Dropout 层：概率为 `0.6`。
- LSTM 层：输出 `256`。
- Dropout 层：概率为 `0.6`。
- 全连接层：输出为 `len(words)`，使用 `softmax` 激活。

使用多分类交叉熵损失函数 `categorical_crossentropy`，同时使用 Adam 优化器，学习率为 `0.001`。

**<font color='red'>挑战</font>：根据上述规定，使用 Keras 序贯模型搭建方法构建 LSTM 模型。**（未提到参数使用默认值）

In [ ]:
from keras import layers
from keras.optimizers import Adam
from keras.models import Sequential

"""定义序贯模型结构
"""

### 代码开始 ### (≈ 6 行代码)
model = Sequential()
model.add(layers.LSTM(512, return_sequences=True, input_shape=(6, len(words))))
model.add(layers.Dropout(0.6))
model.add(layers.LSTM(256))
model.add(layers.Dropout(0.6))
model.add(layers.Dense(len(words), activation='softmax'))
### 代码结束 ###

"""优化器和损失函数
"""

### 代码开始 ### (≈ 2 行代码)
optimizer = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
### 代码结束 ###

**运行测试：**

In [ ]:
model.summary()

**期望输出：**

    _________________________________________________________________
    Layer (type)                 Output Shape              Param #   
    =================================================================
    lstm_1 (LSTM)                (None, 6, 512)            12421120  
    _________________________________________________________________
    dropout_1 (Dropout)          (None, 6, 512)            0         
    _________________________________________________________________
    lstm_2 (LSTM)                (None, 256)               787456    
    _________________________________________________________________
    dropout_2 (Dropout)          (None, 256)               0         
    _________________________________________________________________
    dense_1 (Dense)              (None, 5552)              1426864   
    =================================================================
    Total params: 14,635,440
    Trainable params: 14,635,440
    Non-trainable params: 0
    _________________________________________________________________

### 模型训练

这里的模型结构非常简单，也就是两个 LSTM 层的堆叠。但是，需要训练的参数非常多，已经超过千万。

接下来，就是训练模型的过程。在这部分中，我们会定义多个函数以便达到目的。如果你有兴趣，可以了解每个函数的用途。当然，如果想更快看到结果，直接点击运行就可以了。

In [ ]:
import numpy as np
from keras.callbacks import ModelCheckpoint, LambdaCallback

max_len = 6  # 五言绝句包含标点
poems = files_content.split(']')  # 诗的 list
poems_num = len(poems)  # 诗的总数量

def word2numF(x): return word2num.get(x, len(words) - 1)

def sample(preds):
    exp_preds = np.asarray(preds).astype('float64')
    preds = exp_preds / np.sum(exp_preds)
    pro = np.random.choice(range(len(preds)), 1, p=preds)
    return int(pro.squeeze())

# 根据给出的首个文字，生成五言绝句
def predict_first(char):
    index = np.random.randint(0, poems_num)
    sentence = poems[index][1-max_len:] + char
    generate = str(char)
    generate += _preds(sentence, length=23)
    return generate

# 内部方法，输入 max_len 长度字符串，返回 length 长度的预测值字符串
def _preds(sentence, length=23):
    sentence = sentence[:max_len]
    generate = ''
    for i in range(length):
        pred = _pred(sentence)
        generate += pred
        sentence = sentence[1:]+pred
    return generate

# 内部方法，根据一串输入，返回单个预测字符
def _pred(sentence):
    sentence = sentence[-max_len:]
    x_pred = np.zeros((1, max_len, len(words)))
    for t, char in enumerate(sentence):
        x_pred[0, t, word2numF(char)] = 1.
    preds = model.predict(x_pred, verbose=0)[0]
    next_index = sample(preds)
    next_char = num2word[next_index]

    return next_char

# 生成数据
def data_generator():
    i = 0
    while 1:
        x = files_content[i: i + max_len]
        y = files_content[i + max_len]

        if ']' in x or ']' in y:
            i += 1
            continue

        y_vec = np.zeros(
            shape=(1, len(words)),
            dtype=np.bool
        )
        y_vec[0, word2numF(y)] = 1.0

        x_vec = np.zeros(
            shape=(1, max_len, len(words)),
            dtype=np.bool
        )

        for t, char in enumerate(x):
            x_vec[0, t, word2numF(char)] = 1.0

        yield x_vec, y_vec
        i += 1

# 训练过程中，每 5 个 epoch 打印出当前的学习情况
def generate_sample_result(epoch, logs):
    if (epoch + 1) % 5 == 0:
        # 随机获取某首诗词中的首字用于测试  
        char = poems[np.random.randint(len(poems))][0]
        generate = predict_first(char)
        print("+++++++++++++ 随机测试: {} +++++++++++++".format(char))
        print(generate)
    
# 训练模型
def train(epochs):
    model.fit_generator(
        generator=data_generator(),
        verbose=True,
        steps_per_epoch=32,
        epochs=epochs,
        callbacks=[
            ModelCheckpoint('poetry_model.h5'),
            LambdaCallback(on_epoch_end=generate_sample_result)]
    )


我们设定训练 2000 个 epoch。每一个 epoch 之后模型都会被保存，然后我们从字典中随机挑选出一个字符作为诗的第一个字，用于测试。

In [ ]:
train(epochs=2000)

你会发现，最开始写出来的五言绝句没有正确的格式、且无法正确使用标点。但是，随着迭代次数的增加，测试结果变得越来越好。最后，我们给出 3000 个 epoch 之后，机器以「风」开头创作的 3 首诗：

<div style="text-align: center; color:brown">
风北溪爱风，山树来多白。衣过星殿望，野方林为野。<br>
风思归明月，荷鸟接今方。翼年游得登，雪明将在人。<br>
风南盖酒羽，溪无可临楼。北未晚散野，临可水幽羽。<br>
</div>

---

<div style="color: #999;font-size: 12px;font-style: italic;">*本课程内容，由作者授权实验楼发布，未经允许，禁止转载、下载及非法传播。</div>